In [1]:
# pip install plotly
# pour visualization de BERTopic

# !pip install bertopic

# pip install SentenceTransformer

import time

In [2]:
# pour un ordi avec GPU :

import GPUtil
import torch

In [3]:
torch.cuda.is_available()

True

In [5]:
!set CUDA_PATH=C:\Users\ehess\anaconda3\Lib\site-packages\numba\cuda

import spacy

In [6]:
spacy.require_gpu()

True

In [7]:
spacy.prefer_gpu()

True

In [120]:
# pour un ordi avec GPU
is_using_gpu = spacy.prefer_gpu()
if is_using_gpu:
    print("Using GPU!")
    torch.set_default_tensor_type("torch.cuda.FloatTensor")
    print("GPU Usage")
    GPUtil.showUtilization()

Using GPU!
GPU Usage
| ID | GPU | MEM |
------------------
|  0 |  0% | 86% |


In [9]:
import pandas as pd

In [134]:
df = pd.read_csv("contrib_from_csv.csv", sep = ",", dtype= str)

# sample d'un petit échantillon
df = df.sample(3)

In [135]:
# supprimer les fin de lignes (deux types de codages)
join_breaklines = lambda x : x.replace("\x0b"," ") 
join_breaklinesn = lambda x : x.replace("\n"," ") 

# supprimer les UNK des ce corpus (trois types de codages)
kill_texte_illisible = lambda x : x.replace("texte illisible", "")
kill_illisible = lambda x : x.replace("illisible", "")
kill_illisible_plural = lambda x : x.replace("illisibles", "")
kill_illisible_capslock = lambda x : x.replace("ILLISIBLE", "")

In [136]:
df["clean"] = df["Contribution"].apply(join_breaklines)
df["clean"] = df["clean"].apply(join_breaklinesn)
df["clean"] = df["clean"].apply(kill_texte_illisible)
df["clean"] = df["clean"].apply(kill_illisible)
df["clean"] = df["clean"].apply(kill_illisible_plural)
df["clean"] = df["clean"].apply(kill_illisible_capslock)

In [75]:
t0 = time.time()

df["try"] = df["clean"].apply(remove_stopwords)
t1 = time.time()
print("Total time: {}".format(t1-t0))

False
False
False
False
False
False
False
False
False
False
Total time: 0.0009992122650146484


In [116]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]
final_stopwords_set = set(final_stopwords_list)
def remove_stopwords (text):
    text = list(text)
    print(text)
    buff =[]
    for word in text :
        if word not in final_stopwords_list :
            buff.append(word)
    output = " ".join(buff)
#     text = [word for word in text if word not in final_stopwords_set]
    return output

In [117]:
testdf = pd.DataFrame([["test de les stopwords",1],["de et donc",2],["voir être arbre",5]], columns = ["tes","bien"])

In [118]:
testdf

,tes,bien
0,test de les stopwords,1
1,de et donc,2
2,voir être arbre,5


In [119]:
testdf["tes"].apply(remove_stopwords)

['t', 'e', 's', 't', ' ', 'd', 'e', ' ', 'l', 'e', 's', ' ', 's', 't', 'o', 'p', 'w', 'o', 'r', 'd', 's']
['d', 'e', ' ', 'e', 't', ' ', 'd', 'o', 'n', 'c']
['v', 'o', 'i', 'r', ' ', 'ê', 't', 'r', 'e', ' ', 'a', 'r', 'b', 'r', 'e']


0    t e s t   d e   l e s   s t p w r d s
1                        d e   e t   d n c
2                  v r   ê t r e   r b r e
Name: tes, dtype: object

In [72]:
df["try"]

206601    impôts plus de justice fiscale par augmentatio...
64289     Mardi 11 Décembre 2018 Veuve, ma pension n'aug...
171719    Mr et Mr VERDOT Laurent 10.01.2010 Bonjour, vo...
197115    Où est la république et quelle république ?? q...
140264    - Démocratie et citoyenneté L’Europe au servic...
119831    Voici mes subjections que j'aimerai voir appli...
29604     DOLÉANCES I.S.F : insister ? Si fait I Monsieu...
176874    transition énergétique est un éclairage public...
111099                                          {{TABLEAU}}
148098    Que les gilets jaunes mettent en place un gouv...
Name: try, dtype: object

In [137]:
df["clean"]

218083    20 ans, l'âge de la retraite ?. Dommage, c'est...
7427      AUTRES THÉMATIQUES D'autres sujets émergent au...
121915    le 24.01.2019 - Baisse de TVA par les produits...
Name: clean, dtype: object

In [138]:
# test sur le GPU

def lemmatize(doc):
    '''
    Performs lemmization of input documents.
    Args:
    - docs: one column of a dataframe
    Output:
    - list (nb of contributions) of list (one cell,(size 2: with position input, lemmatized input)) of list of strings
    '''

    spacy.require_gpu()
    nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
    nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
    
    lemma_text_list = []  
    lemma_pos_list = []
    t0 = time.time()
    for doc in nlp.pipe(doc):
#         buffer = []
#         lemma_text_list.append([[" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]])     
#         buffer.append([" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]) 
#         print(buffer)
#        on ne join() pas à ce moment là car la list reste nécessaire pour enlever les stopwords.
        lemma_text_list.append([x.pos_ for x in doc])
        lemma_pos_list.append([x.lemma_ for x in doc])

    t1 = time.time()
    print("Total time: {}".format(t1-t0))

    return (lemma_text_list, lemma_pos_list)

In [139]:
# GPU Total time: 130.2461621761322 (without if and with fr_core_news_md)
lemmatized_pos, lemmatized_lemma = lemmatize(df["clean"])

Total time: 0.6529309749603271


In [140]:
df["lemmatized_text"] = lemmatized_lemma

In [141]:
df["lemmatized_pos"] = lemmatized_pos

In [20]:
# pour afficher l'ensemble des cells, à n'utiliser que pour afficher une portion seulement

# pd.set_option('display.max_colwidth', None)

In [21]:
# spacy.require_cpu()

In [22]:
# test CPU

# def lemmatize(doc, allowed_postags = ["NOUN", "ADJ", "VERB", "ADV"]):
#     nlp = spacy.load("fr_core_news_md", disable = ["parser", "ner"])
#     nlp.max_length = 3000000 #this is possible as we're not using parser or ner.
#     lemma_text_list = []
#     t0 = time.time()
#     for doc in nlp.pipe(doc):
#         lemma_text_list.append([[" ".join(x.pos_ for x in doc)], [" ".join(x.lemma_ for x in doc)]])      
#     t1 = time.time()
#     print("Total time: {}".format(t1-t0))  # ~9.759 seconds on 5000 rows
#     return (lemma_text_list)

In [151]:
# lemmatized = lemmatize(df["clean"])
# CPU Total time: 160.70105290412903 without if and with fr_core_news_md
df

,Catégorie,Date de réception,Code postal,Code INSEE,Numéro d'ordre arbitraire,Type Graphie TT,Numéro de page,Numéro séquentiel,Contribution,joined_id,clean,lemmatized_text,lemmatized_pos
218083,CC,190315,44980,44172,16802,MD,161,76,"20 ans, l'âge de la retraite ?. Dommage, c'est...",CC_44980_190315_44172_MD_16802,"20 ans, l'âge de la retraite ?. Dommage, c'est...","[20, an, ,, le, âge, de, le, retraite, ?, ., d...","[NUM, NOUN, PUNCT, DET, NOUN, ADP, DET, NOUN, ..."
7427,CC,190225,37110,37116,01220,D,51,21,AUTRES THÉMATIQUES D'autres sujets émergent au...,CC_37110_190225_37116_MD_01220,AUTRES THÉMATIQUES D'autres sujets émergent au...,"[autre, thématique, de, autre, sujet, émerger,...","[ADJ, ADJ, ADP, ADJ, NOUN, VERB, ADP, NOUN, AD..."
121915,CC,190225,30300,30117,05247,MD,25,6,le 24.01.2019\n- Baisse de TVA par les produit...,CC_30300_190225_30117_MD_05247,le 24.01.2019 - Baisse de TVA par les produits...,"[le, 24.01.2019, -, baisse, de, tva, par, le, ...","[DET, PROPN, PUNCT, NOUN, ADP, NOUN, ADP, DET,..."


In [154]:
doc = df.lemmatized_text.tolist()
def remove_stopwords(text):
    stopwords = final_stopwords_list
    text = [word for word in text if word not in stopwords]
    return text

test = remove_stopwords(doc)
print(doc[0])

['20', 'an', ',', 'le', 'âge', 'de', 'le', 'retraite', '?', '.', 'dommage', ',', 'ce', 'être', 'avoir', 'ce', 'âge', 'que', 'tu', 'te', 'être', 'ouvrir', 'au', 'monde', '.', 'tu', 'ne', 'avoir', 'pas', 'le', 'droit', 'avoir', 'un', 'scolarisation', '.', 'pourtant', ',', 'le', 'progrès', 'de', 'neuroscience', '.', 'montrer', 'le', 'plasticité', 'cérébral', '.', 'tu', 'en', 'être', 'le', 'preuve', 'vivant', '.', 'le', 'motivation', 'sans', 'faille', 'de', 'ton', 'mère', '.', 'est', 'bien', 'récompenser', 'par', 'ton', 'progrès', '.', 'progrès', ',', 'réel', 'source', 'de', 'espoir', '.', 'dans', 'un', 'quotidien', 'trop', 'souvent', 'anxyogèn', '.', 'mon', 'fils', ',', 'quel', 'bataille', 'pour', 'te', 'aider', 'à', 'te', 'épanouir', '.', 'et', 'te', 'permettre', 'de', 'rejoindre', 'le', 'communauté', 'humain', '.', 'autisme', ',', 'en', '2019', ',', 'en', 'France', ',']


In [147]:
from spacy.lang.fr.stop_words import STOP_WORDS as fr_stop
from spacy.lang.en.stop_words import STOP_WORDS as en_stop

final_stopwords_list = list(fr_stop) + list(en_stop) + ["\n"]
final_stopwords_set = set(final_stopwords_list)
def remove_stopwords (text):

    text = [print(word) for word in text] #if word not in final_stopwords_set]

    return text

In [148]:
t0 = time.time()

nostopword = remove_stopwords(df["lemmatized_text"])

t1 = time.time()
print("Total time: {}".format(t1-t0))

['20', 'an', ',', 'le', 'âge', 'de', 'le', 'retraite', '?', '.', 'dommage', ',', 'ce', 'être', 'avoir', 'ce', 'âge', 'que', 'tu', 'te', 'être', 'ouvrir', 'au', 'monde', '.', 'tu', 'ne', 'avoir', 'pas', 'le', 'droit', 'avoir', 'un', 'scolarisation', '.', 'pourtant', ',', 'le', 'progrès', 'de', 'neuroscience', '.', 'montrer', 'le', 'plasticité', 'cérébral', '.', 'tu', 'en', 'être', 'le', 'preuve', 'vivant', '.', 'le', 'motivation', 'sans', 'faille', 'de', 'ton', 'mère', '.', 'est', 'bien', 'récompenser', 'par', 'ton', 'progrès', '.', 'progrès', ',', 'réel', 'source', 'de', 'espoir', '.', 'dans', 'un', 'quotidien', 'trop', 'souvent', 'anxyogèn', '.', 'mon', 'fils', ',', 'quel', 'bataille', 'pour', 'te', 'aider', 'à', 'te', 'épanouir', '.', 'et', 'te', 'permettre', 'de', 'rejoindre', 'le', 'communauté', 'humain', '.', 'autisme', ',', 'en', '2019', ',', 'en', 'France', ',']
['autre', 'thématique', 'de', 'autre', 'sujet', 'émerger', 'au', 'sein', 'de', 'doléance', 'analyser', '.', 'si', 'le'

In [ ]:
nostopword[:3]